## Neural Network

This jupyter notebook implements the neural network model using convolutional layers

In [5]:
#Import libraries

from torchvision.io import read_image
from torch.utils.data import DataLoader
import torchvision.transforms as T
import torch 
import torch.nn as nn # basic building block for neural neteorks
import torch.nn.functional as F # import convolution functions like Relu
import torch.optim as optim # optimzer
import os

from XRayDataset import XRayDataset
from dataloader import transforms, resize, train_path, test_path, val_path

In [2]:
# MY MODEL EXPECTS AN 512 x 512 IMAGE with dtype = float32 //IAN's comment
#JP: Still not sure in which parts it expects a 512x512

class CustomNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.LeNet = nn.Sequential(     
            # convolutional layers            
            nn.Sequential(                                            # FIRST LAYER: (INPUT LAYER)
              nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),    # CONVOLUTION 
              nn.BatchNorm2d(6),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size = 2, stride = 2)),             # POOLING
            nn.Sequential(                                            # SECOND LAYER: HIDDEN LAYER 1
              nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),   # CONVOLUTION 
              nn.BatchNorm2d(16),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size = 2, stride = 2)),             # POOLING
            # fully connected layers
            nn.Flatten(),
            nn.Linear(250000, 120),                                   # THIRD LAYER: LINEAR YEAR, HIDDEN LAYER 2
            nn.ReLU(),                                                # HIDDEN LAYER's ACTIVATION FUNCION
            nn.Linear(120, 84),                                       # FOURTH LAYER: LINEAR YEAR, HIDDEN LAYER 3
            nn.Sigmoid(),                                                # HIDDEN LAYER's ACTIVATION FUNCION
            ##I changed the ReLU activation to sigmoid following https://towardsdatascience.com/how-to-choose-the-right-activation-function-for-neural-networks-3941ff0e6f9c
            # output layer
            nn.Linear(84, 2)                                          # OUTPUT LAYER
        )

    def forward(self, x):
        out = self.LeNet(x)
        return out

In [3]:
model = CustomNeuralNetwork()

In [14]:
#Dataloader code from Checkpoint #1

training_data = XRayDataset(train_path, transforms)
val_data = XRayDataset(val_path, resize)
test_data = XRayDataset(test_path, resize)

#Discussion with Ian said to start with small batches (2,4)
#Medium post said do powers of 2 starting with 16 
#I'll go with middle point and do batch = 8
#https://medium.com/data-science-365/determining-the-right-batch-size-for-a-neural-network-to-get-better-and-faster-results-7a8662830f15

batch_s = 8

train_dataloader = DataLoader(training_data, batch_size=batch_s, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_s, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_s, shuffle=True)


[tensor([[[[0.1882, 0.1922, 0.1882,  ..., 0.0784, 0.0784, 0.0784],
           [0.2000, 0.2353, 0.2000,  ..., 0.0784, 0.0784, 0.0784],
           [0.1922, 0.2196, 0.2000,  ..., 0.0784, 0.0784, 0.0784],
           ...,
           [0.1725, 0.1725, 0.1490,  ..., 0.0941, 0.0941, 0.0941],
           [0.1725, 0.1725, 0.1451,  ..., 0.0941, 0.0941, 0.0941],
           [0.1765, 0.1725, 0.1765,  ..., 0.0941, 0.0941, 0.0941]]],
 
 
         [[[0.5647, 0.5725, 0.5647,  ..., 0.9922, 1.0000, 1.0000],
           [0.5686, 0.5686, 0.5608,  ..., 0.9961, 1.0000, 1.0000],
           [0.5804, 0.5725, 0.5647,  ..., 0.9961, 1.0000, 1.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.1922, 0.2157, 0.2314],
           [0.0000, 0.0000, 0.0000,  ..., 0.1961, 0.2275, 0.2471],
           [0.0000, 0.0000, 0.0000,  ..., 0.2039, 0.2275, 0.2392]]],
 
 
         [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000

In [7]:
# 3: Define a Loss function and optimizer
criterion = nn.NLLLoss() #a.Change the cross entropy of the original code to NLL 
#which we have used before for binary classification
#b. 
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [10]:
# 4: Train and validate the network
EPOCHS = 100

train_losses = []
train_accuracies = []
val_losses = []
val_accuarcies = []

for _ in range(EPOCHS):  # loop over the dataset multiple times

    # TRAIN
    # Make sure gradient tracking is on, and do a pass over the data
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_dataloader):
      # get the inputs; data is a list of [inputs, labels]
      inputs, labels = data  

      # zero the parameter gradients
      optimizer.zero_grad()

      # forward + backward + optimize
      outputs = model(inputs) #Having a problem here!
      loss = criterion(m(outputs), labels) ##NLL requires to pass the outputs through m(.) / delete in case of cross entropy
      loss.backward()
      optimizer.step()

      # keep track of the loss
      running_loss += loss.item()
      # ALSO CALCULATE YOUR ACCURACY METRIC
      
    avg_train_acc = 0
    avg_train_loss = running_loss / (i + 1)     # i + 1 gives us the total number of batches in train dataloader
    # CALCULATE AVERAGE ACCURACY METRIC
    train_losses.append(avg_train_loss)
    train_accuracies.append(avg_train_acc)

    #VALIDATE
    # in the validation part, we don't want to keep track of the gradients 
    model.eval() 
    for i, data in enumerate(val_dataloader):
      inputs, labels = data 
      outputs = model(inputs)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (8x1082736 and 250000x120)

In [ ]:
#5. Test model
with torch.no_grad():
    total = 0
    correct = 0
    for i, data in enumerate(test_dataloader):
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        acc = correct / total
    